In [4]:
import os 
import pandas as pd 
import shutil
import numpy as np
import pickle
from xgboost.sklearn import XGBRegressor
from scipy.stats import norm
import time

In [10]:
for i in range(1,8314):
    os.chdir(r'D:\Google Drive\Research\Ph.D. Topic Research\Cost_Benefit\UCERF2_Event_Processed_Data\Event%i'%i)
    shutil.copy('Building Site Map.csv', r'C:\Users\User\Desktop\Optimization\Evaluation\Events\Event%i.csv'%i)
    # Sa = pd.read_csv('Building Site Map.csv', header = None)
    # Sa.to_csv(r'C:\Users\User\Desktop\Optimization\Evaluation\Events\Event%i.csv'%i, header = None)

In [2]:
def summary_fatality(percentage_miss_strength, sa):
    '''
    This function is developed for compute the total strength added by algorithm
    '''
    im_bin = [0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, float('inf')]
    retrof_ratio = {}
    prop = {}
    retrof_ds_ratio = {}
    retrof_ds_median = {}
    for i in range(1,9):
        temp = pd.DataFrame() 
        temp['Index'] = existing_prop['Index']
        temp['Xpeakstrength'] = np.divide(np.multiply(percentage_miss_strength[i-1], missing_strength['X Diff']) + peak_strength['Xpeakstrength'], peak_strength['Xpeakstrength'])
        temp['Zpeakstrength'] = np.divide(np.multiply(percentage_miss_strength[i-1], missing_strength['Z Diff']) + peak_strength['Zpeakstrength'], peak_strength['Zpeakstrength'])
        temp = pd.concat([temp,retrof_upper_limit]).min(level=0)
        retrof_ratio[i] = temp
        prop[i] = existing_prop.drop(columns=['Xpeakstrength', 'Zpeakstrength'])
        prop[i]['Xpeakstrength'], prop[i]['Zpeakstrength'] = retrof_ratio[i]['Xpeakstrength'], retrof_ratio[i]['Zpeakstrength']

        retrof_ds_ratio[i] = pd.DataFrame(data = prop[i]['Index'])
        retrof_ds_ratio[i]['complete_ratio'] = retrof_complete_ratio.predict(prop[i].iloc[:,1:])
        # Compute retrofitted buildings damage state medians 
        retrof_ds_median[i] = pd.DataFrame(data = np.multiply(retrof_ds_ratio[i]['complete_ratio'],existing_ds['Complete']))
        retrof_ds_median[i].insert(0,'Index', retrof_ds_ratio[i]['Index'])

    region_fatality = compute_fatality(sampled_building_info, total_fatality, retrof_ds_median, sa_simulated, sa, im_bin)
    return region_fatality

def compute_fatality(sampled_building_info, building_fatality, building_dsm, design_Sa, Sa_map, IM_bin):
    '''    
    This function is mainly used for computing regional earthquake losses given scenario
    sampled_building_info: assigned missing building information 
    building_loss: prototype archetype losses in each damage state 
    building_dsm: retrofitted prototype archetype damage state medians 
    Sa_map: ground shaking maps given scenario/event
    IM_bin: different retrofit strength is applied to different intensity bins
    '''

    buildings = pd.DataFrame(data = sampled_building_info['Index'])
    # Building collapse  at each location
    site_fatality = buildings.merge(total_fatality, how='left')

    # Compute probability of complete damage at each site given each shake map
    ds_prob_map = pd.DataFrame(np.zeros(Sa_map.shape))

    # Loop over all shake maps
    for k in range(8):
        for j in range(Sa_map.shape[1]):
            sub_Sa_map = Sa_map.loc[(design_Sa.iloc[:,0] < IM_bin[k+1]) & (design_Sa.iloc[:,0] >= IM_bin[k]), :].iloc[:,j]
            if sub_Sa_map.shape[0] == 0: 
                continue
            sub_building_map = buildings.loc[(design_Sa.iloc[:,0] < IM_bin[k+1]) & (design_Sa.iloc[:,0] >= IM_bin[k]),:]
            building_ds_median = sub_building_map.merge(building_dsm[k+1])
            ds_prob_map.loc[(design_Sa.iloc[:,0] < IM_bin[k+1]) & (design_Sa.iloc[:,0] >= IM_bin[k]), j] = norm.cdf(np.log(sub_Sa_map.astype('float64')), loc = np.log(building_ds_median.iloc[:,1]), scale = 0.6)
    collapse_prob = ds_prob_map.mean(axis = 1)

    portfolio_fatality = np.multiply(collapse_prob,site_fatality.iloc[:,1]).sum().sum()
    return portfolio_fatality

    
def ordinance_fatality_representative_events(sampled_building_info, building_fatality, building_dsm, Sa_map):
    '''    
    This function is mainly used for computing regional earthquake losses given scenario
    sampled_building_info: assigned missing building information 
    building_loss: prototype archetype losses in each damage state 
    building_dsm: retrofitted prototype archetype damage state medians 
    Sa_map: ground shaking maps given scenario/event
    '''
    buildings = pd.DataFrame(data = sampled_building_info['Index'])
    # Building collapse  at each location
    site_fatality = buildings.merge(building_fatality)

    # Building damage state median at each location 
    building_ds_median = buildings.merge(building_dsm, how = 'left')

    # Compute probability of complete damage at each site given each shake map
    ds_prob_map = pd.DataFrame(np.zeros(Sa_map.shape))
    # Loop over all shake maps
    for j in range(Sa_map.shape[1]):
        ds_prob_map.iloc[:,j] = norm.cdf(np.log(Sa_map.iloc[:,j]), loc = np.log(building_ds_median.iloc[:,1].astype('float')), scale = 0.6)
    collapse_prob = ds_prob_map.mean(axis = 1)

    portfolio_fatality = np.multiply(collapse_prob,site_fatality.iloc[:,1]).sum().sum()
    
    return portfolio_fatality



In [3]:
BaseDirectory = r'C:\Users\User\Desktop\Optimization\Evaluation'
os.chdir(BaseDirectory)
retrof_complete_ratio = pickle.load(open('Retrof_Complete_Ratio.pkl','rb'))

sampled_building_info = pd.read_csv('buildinginfo.csv')
sa_simulated = pd.read_csv('Sa_simulated_real.csv', header = None)

# Read existing building damage state
existing_ds = pd.read_csv('building_ds.csv')

# Read existing building properties and pushover 
existing_prop = pd.read_csv('building_performance.csv')

# Read building configuration, which is mainly used for loss assessment 
building_config = pd.read_csv('building_configuration.csv')

# Read building fatality 
# total_fatality = pd.read_csv('fatality_rate.csv')
# Use complete loss as criteria: complete loss = building collapse loss + fatality loss ($2 million per fatality)
total_fatality = pd.read_csv('complete_loss.csv') 

# Read retrofit upper limit, which corresponding to NON SWOF building peak strength
retrof_upper_limit = pd.read_csv('UpperLimit.csv')

# Read missing strength
missing_strength = pd.read_csv('building_miss_strength.csv')

# Read existing building damage state medians
existing_ds = pd.read_csv('building_ds.csv')  

peak_strength = pd.DataFrame()
peak_strength['Index'] = existing_prop['Index']
peak_strength['Xpeakstrength'] = np.multiply(existing_prop['SeismicWeight'], existing_prop['Xpeakstrength'])
peak_strength['Zpeakstrength'] = np.multiply(existing_prop['SeismicWeight'], existing_prop['Zpeakstrength'])

ordinance_ds = pd.read_csv(r'ordinance_building_ds.csv')
ordinance_ds = ordinance_ds.drop(columns=['Slight', 'Moderate', 'Severe'])

[17:00:48] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [8]:
algorithm_best = [0.5373147823253948, 0.6776796686980564, 0.8108382470888009,
       0.8537486161975294, 0.8639264114170233, 0.7609222356570593,
       0.710229536829122, 0.5536388483053063]
algorithm_loss, ordinance_loss, existing_loss = [], [], []
s = time.time()
for i in range(1,8313):
    os.chdir(r'C:\Users\User\Desktop\Optimization\Evaluation\Events')
    Sa = pd.read_csv('Event%i.csv'%i, header = None)
    algorithm_loss.append(summary_fatality(algorithm_best, Sa))
    ordinance_loss.append (ordinance_fatality_representative_events(sampled_building_info, total_fatality, ordinance_ds, Sa))
    existing_loss.append(ordinance_fatality_representative_events(sampled_building_info, total_fatality, existing_ds.drop(columns=['Slight', 'Moderate', 'Severe']), Sa))
    print(i, time.time()-s)
r = pd.DataFrame(data=[existing_loss,ordinance_loss,algorithm_loss])


7452 51130.552067279816
7453 51137.46754693985
7454 51144.444902181625
7455 51151.239716529846
7456 51158.15522098541
7457 51165.16048836708
7458 51172.09294652939
7459 51178.97653889656
7460 51185.89004921913
7461 51192.784611701965
7462 51199.119669914246
7463 51205.41784930229
7464 51211.8077378273
7465 51218.115916490555
7466 51224.43099927902
7467 51230.706218481064
7468 51236.94353747368
7469 51243.33843612671
7470 51249.73133802414
7471 51256.05742049217
7472 51262.34263896942
7473 51268.711606025696
7474 51274.99477529526
7475 51281.322853565216
7476 51287.759639024734
7477 51294.034856557846
7478 51300.34301877022
7479 51306.70098924637
7480 51312.974272727966
7481 51319.252509355545
7482 51325.578592062
7483 51331.847829818726
7484 51338.15595793724
7485 51344.431176662445
7486 51350.71736621857
7487 51357.08234381676
7488 51363.306696653366
7489 51369.7584438324
7490 51376.15533542633
7491 51382.470447301865
7492 51388.87232708931
7493 51395.18743777275
7494 51401.534462928

In [12]:
r.to_csv(r'C:\Users\User\Desktop\Optimization\CollapseLoss_DBE\EvaluationResults.csv')